#### 初回のみ実行

```julia
`sudo apt install mecab mecab-ipadic-utf8` |> run
```

In [1]:
using Pkg
Pkg.add(["Awabi", "JSON", "UUIDs", "IJulia"])
using Awabi, JSON, UUIDs, IJulia

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
text = """
# 自然言語分析

## ワードクラウド

テキストの中で頻出する単語を抽出し、出現頻度に応じた大きさでビジュアル化

話題や主要なテーマを一目で把握するのに効果的

## 共起ネットワーク

単語間の共起関係をネットワーク図で表現

単語のつながりや文脈を視覚的に理解できる

## 感情分析の色分けテキスト

文章中の各文や段落をその感情 (ポジティブ・ネガティブ) に基づいて色分け

全体的な感情の流れや変化が把握しやすい

## トピックのサンバースト図

トピックモデルの結果を同心円状のサンバースト図で表現

トピック間の階層構造が視覚的に明確になる

## 固有表現の強調表示

人名、地名、組織名など固有表現を抽出し、色やラベルで強調表示

テキスト中の重要な entities がわかりやすくなる

## 文の類似度ヒートマップ

文書内の文同士の類似度を計算し、ヒートマップで可視化

文書構造や話題の遷移を理解する手がかりになる

## 品詞の比率円グラフ

名詞、動詞、形容詞などの品詞の割合を円グラフで表示

テキストのスタイルや文体の特徴が見えてくる

## 単語ベクトルの次元削減プロット

word2vecなどで単語をベクトル化し、次元削減して2Dにプロット

単語の意味的な近さを視覚的に捉えられる
"""

"# 自然言語分析\n\n## ワードクラウド\n\nテキストの中で頻出する単語を抽出し、出現頻度に応じた大きさでビジュアル化\n\n話題や主要なテーマを一目で把握するのに効果的\n\n## 共起ネットワーク\n\n単語間の共起関係をネットワーク図で表現\n\n単語のつながりや文脈を視覚的に理解できる\n\n## 感情分析の色分けテキスト\n\n文章中の各文や段落をその感情 (ポジティブ・ネガティブ) に基づいて色分け\n\n全体的な感情の流れや変化が把握しやすい\n\n## トピックのサンバースト図\n\nトピックモデルの結果を同心円状のサンバースト図で表現\n\nトピック間の階層構" ⋯ 67 bytes ⋯ "人名、地名、組織名など固有表現を抽出し、色やラベルで強調表示\n\nテキスト中の重要な entities がわかりやすくなる\n\n## 文の類似度ヒートマップ\n\n文書内の文同士の類似度を計算し、ヒートマップで可視化\n\n文書構造や話題の遷移を理解する手がかりになる\n\n## 品詞の比率円グラフ\n\n名詞、動詞、形容詞などの品詞の割合を円グラフで表示\n\nテキストのスタイルや文体の特徴が見えてくる\n\n## 単語ベクトルの次元削減プロット\n\nword2vecなどで単語をベクトル化し、次元削減して2Dにプロット\n\n単語の意味的な近さを視覚的に捉えられる\n"

In [3]:
tokens = tokenize(Tokenizer(), text)

266-element Vector{Tuple{String, String}}:
 ("#", "名詞,サ変接続,*,*,*,*,*")
 ("自然", "名詞,形容動詞語幹,*,*,*,*,自然,シゼン,シゼン")
 ("言語", "名詞,一般,*,*,*,*,言語,ゲンゴ,ゲンゴ")
 ("分析", "名詞,サ変接続,*,*,*,*,分析,ブンセキ,ブンセキ")
 ("##", "名詞,サ変接続,*,*,*,*,*")
 ("ワードクラウド", "名詞,一般,*,*,*,*,*")
 ("テキスト", "名詞,一般,*,*,*,*,テキスト,テキスト,テキスト")
 ("の", "助詞,連体化,*,*,*,*,の,ノ,ノ")
 ("中", "名詞,非自立,副詞可能,*,*,*,中,ナカ,ナカ")
 ("で", "助詞,格助詞,一般,*,*,*,で,デ,デ")
 ⋮
 ("な", "助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ")
 ("近", "形容詞,自立,*,*,形容詞・アウオ段,ガル接続,近い,チカ,チカ")
 ("さ", "名詞,接尾,特殊,*,*,*,さ,サ,サ")
 ("を", "助詞,格助詞,一般,*,*,*,を,ヲ,ヲ")
 ("視覚", "名詞,一般,*,*,*,*,視覚,シカク,シカク")
 ("的", "名詞,接尾,形容動詞語幹,*,*,*,的,テキ,テキ")
 ("に", "助詞,副詞化,*,*,*,*,に,ニ,ニ")
 ("捉え", "動詞,自立,*,*,一段,未然形,捉える,トラエ,トラエ")
 ("られる", "動詞,接尾,*,*,一段,基本形,られる,ラレル,ラレル")

In [4]:
words  = filter(token -> occursin(r"名詞.*", token[2]), tokens)
counts = Dict{String, Int}()
for word in words
    counts[word[1]] = get(counts, word[1], 0) + 1
end
res = sort(collect(counts), by = tuple -> last(tuple), rev=true)

94-element Vector{Pair{String, Int64}}:
       "##" => 8
       "的" => 6
     "単語" => 6
     "表現" => 4
 "テキスト" => 4
     "視覚" => 3
 "トピック" => 3
       "化" => 3
       "文" => 3
       "中" => 3
            ⋮
     "word" => 1
     "同士" => 1
   "モデル" => 1
       "名" => 1
     "出現" => 1
     "動詞" => 1
     "一目" => 1
 "つながり" => 1
     "階層" => 1

In [5]:
function wordcloud(res)
    divid = "d3" * string(UUIDs.uuid4())
    spec = JSON.json(res)

    html = """
      <svg id="$divid" width="800" height="800"></svg>
      <script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js"></script>
      <script type="text/javascript">
          require.config({
              paths: {
                  d3: "https://cdnjs.cloudflare.com/ajax/libs/d3/5.16.0/d3.min",
                  cloud: "https://cdnjs.cloudflare.com/ajax/libs/d3-cloud/1.2.5/d3.layout.cloud.min",
              }
          });
          require(["d3", "cloud"], (d3, cloud) => {
              const data       = $spec;
              const countMax   = d3.max(data, (d) => d[Object.keys(d)[0]]);
              const sizeScale  = d3.scaleLinear().domain([0, countMax]).range([10, 100]);
              const colorScale = d3.scaleOrdinal(d3.schemeCategory10);
              const words      = data.map((d) => ({
                  text: Object.keys(d)[0],
                  size: sizeScale(d[Object.keys(d)[0]]) // 頻出カウントを文字サイズに反映
              }));
              cloud().size([800, 800])
                  .words(words)
                  .rotate(() => ((1.2 - Math.random()) | 0) * 90)
                  .fontSize((d) => d.size)
                  .on("end", draw)
                  .start();
              function draw(words) {
                  d3.selectAll('text').remove();
                  d3.select("#$divid")
                      .attr("width", 800)
                      .attr("height", 800)
                      .append("g")
                      .attr("transform", "translate(400,400)")
                      .selectAll("text")
                      .data(words)
                      .enter().append("text")
                      .style("font-size", (d) => d.size + "px")
                      .style("fill", (d, i) => colorScale(i))
                      .attr("text-anchor", "middle")
                      .attr("transform", (d) => "translate(" + [d.x, d.y] + ")rotate(" + d.rotate + ")")
                      .text((d) => d.text);
              }
          });
      </script>
    """
end

wordcloud (generic function with 1 method)

In [6]:
HTML(wordcloud(res))

HTML{String}("  <svg id=\"d38c4165eb-54b3-445c-8f11-1444b9fe28c2\" width=\"800\" height=\"800\"></svg>\n  <script src=\"https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js\"></script>\n  <script type=\"text/javascript\">\n      require.config({\n          paths: {\n              d3: \"https://cdnjs.cloudflare.com/ajax/libs/d3/5.16.0/d3.min\",\n              cloud: \"https://cdnjs.cloudflare.com/ajax/libs/d3-cloud/1.2.5/d3.layout.cloud.min\",\n          }\n      });\n      require([\"d3\", \"cloud\"], (d3, cloud) => {\n          const data = [{\"##\":8},{\"的\":6},{\"単語\":6},{\"表現\":4},{\"テキスト\":4},{\"視覚\":3},{\"トピック\":3},{\"化\":3},{\"文\":3},{\"中\":3},{\"表示\":3},{\"図\":3},{\"感情\":3},{\"2\":2},{\"円グラフ\":2},{\"ネットワーク\":2},{\"文書\":2},{\"固有\":2},{\"抽出\":2},{\"共起\":2},{\"分析\":2},{\"話題\":2},{\"把握\":2},{\"間\":2},{\"次元\":2},{\"強調\":2},{\"度\":2},{\"ヒートマップ\":2},{\"色分け\":2},{\"プロット\":2},{\"サンバースト\":2},{\"類似\":2},{\"理解\":2},{\"さ\":2},{\"構造\":2},{\"削減\":2},{\"ベクトル\":2},{\"品詞\":2},{\"vec\":1},{\"entities\":1},{\"各\":1},{\"効果\":1},{\"全体\":1},{\"#\":1},{\"組織\":1},{\"特徴\":1},{\"ポジティブ・ネガティブ\":1},{\"色\":1},{\"名詞\":1},{\")\":1},{\"頻度\":1},{\"(\":1},{\"頻出\":1},{\"重要\":1},{\"割合\":1},{\"遷移\":1},{\"結果\":1},{\"テーマ\":1},{\"状\":1},{\"段落\":1},{\"同心円\":1},{\"変化\":1},{\"文章\":1},{\"意味\":1},{\"言語\":1},{\"関係\":1},{\"スタイル\":1},{\"文体\":1},{\"D\":1},{\"可視\":1},{\"地名\":1},{\"流れ\":1},{\"ビジュアル\":1},{\"明確\":1},{\"自然\":1},{\"比率\":1},{\"文脈\":1},{\"内\":1},{\"計算\":1},{\"ラベル\":1},{\"主要\":1},{\"ワードクラウド\":1},{\"形容詞\":1},{\"手がかり\":1},{\"人名\":1},{\"word\":1},{\"同士\":1},{\"モデル\":1},{\"名\":1},{\"出現\":1},{\"動詞\":1},{\"一目\":1},{\"つながり\":1},{\"階層\":1}];\n          const countMax = d3.max(data, (d) => d[Object.keys(d)[0]]);\n          const sizeScale = d3.scaleLinear().domain([0, countMax]).range([10, 100]);\n          const colorScale = d3.scaleOrdinal(d3.schemeCategory10);\n          const words = data.map((d) => ({\n              text: Object.keys(d)[0],\n              size: sizeScale(d[Object.keys(d)[0]]) // 頻出カウントを文字サイズに反映\n          }));\n          cloud().size([800, 800])\n              .words(words)\n              .rotate(() => ((1.2 - Math.random()) | 0) * 90)\n              .fontSize((d) => d.size)\n              .on(\"end\", draw)\n              .start();\n          function draw(words) {\n              d3.selectAll('text').remove();\n              d3.select(\"#d38c4165eb-54b3-445c-8f11-1444b9fe28c2\")\n                  .attr(\"width\", 800)\n                  .attr(\"height\", 800)\n                  .append(\"g\")\n                  .attr(\"transform\", \"translate(400,400)\")\n                  .selectAll(\"text\")\n                  .data(words)\n                  .enter().append(\"text\")\n                  .style(\"font-size\", (d) => d.size + \"px\")\n                  .style(\"fill\", (d, i) => colorScale(i))\n                  .attr(\"text-anchor\", \"middle\")\n                  .attr(\"transform\", (d) => \"translate(\" + [d.x, d.y] + \")rotate(\" + d.rotate + \")\")\n                  .text((d) => d.text);\n          }\n      });\n  </script>\n")

### 結果（github上だとjsが実行されてない）
![image.png](image.png)